In [15]:
import timm

ckpt = "/home/jin/code/MADE/weights/eva02_base_patch14_224/model.safetensors"

# 这个名字要用 timm 里“带预训练tag”的那个（最常见是 in22k_in1k 这类）
model_name = "eva02_base_patch14_224.mim_in22k"

model = timm.create_model(
    model_name,
    pretrained=True,  # 让它拿到 pretrained_cfg（预处理配置等）
    pretrained_cfg_overlay=dict(file=ckpt),  # 关键：权重改成本地文件
    num_classes=0,    # 做特征抽取/REID 一般设 0（去掉分类头）
)
model.eval()


Eva(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (rope): RotaryEmbeddingCat()
  (norm_pre): Identity()
  (blocks): ModuleList(
    (0-11): 12 x EvaBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): EvaAttention(
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): SwiGLU(
        (fc1_g): Linear(in

In [16]:
import timm
import torch
from safetensors.torch import load_file

ckpt = "/home/jin/code/MADE/weights/eva02_base_patch14_224/model.safetensors"
model_name = "eva02_base_patch14_224.mim_in22k"

model = timm.create_model(model_name, pretrained=False, num_classes=0)

state = load_file(ckpt)  # safetensors 专用读取
missing, unexpected = model.load_state_dict(state, strict=False)

print("missing:", len(missing))
print("unexpected:", len(unexpected))
print("example missing:", missing[:10])
print("example unexpected:", unexpected[:10])


missing: 0
unexpected: 0
example missing: []
example unexpected: []


In [18]:
import torch

w = model.patch_embed.proj.weight.detach().float()
print("patch_embed.proj.weight mean/std:", w.mean().item(), w.std().item())


patch_embed.proj.weight mean/std: -0.0001963048562174663 0.017250338569283485


In [21]:
import timm
import torch
from safetensors.torch import load_file

ckpt = "/home/jin/code/MADE/weights/eva02_base_patch14_224/model.safetensors"
model_name = "eva02_base_patch14_224.mim_in22k"

# 1) 随机初始化模型（不加载任何权重）
torch.manual_seed(12345)           # 固定一下，防止你环境里每次seed被重置
m_rand = timm.create_model(model_name, pretrained=False, num_classes=0)
w0 = m_rand.patch_embed.proj.weight.detach().cpu()

# 2) 加载ckpt模型
torch.manual_seed(12345)           # 故意用同一个seed，避免“初始化差异”影响判断
m_load = timm.create_model(model_name, pretrained=False, num_classes=0)
state = load_file(ckpt)
m_load.load_state_dict(state, strict=True)
w1 = m_load.patch_embed.proj.weight.detach().cpu()

# 3) 逐元素对比（最可靠）
diff = (w0 - w1).abs()
print("max abs diff:", diff.max().item())
print("mean abs diff:", diff.mean().item())
print("allclose:", torch.allclose(w0, w1))


max abs diff: 0.19446143507957458
mean abs diff: 0.024049267172813416
allclose: False


In [1]:
from safetensors.torch import load_file
sd = load_file("/home/jin/code/MADE/weights/eva02_large_patch14_clip_224/open_clip_model.safetensors")
ks = list(sd.keys())
print(ks[:30])


['logit_scale', 'text.ln_final.bias', 'text.ln_final.weight', 'text.positional_embedding', 'text.text_projection', 'text.token_embedding.weight', 'text.transformer.resblocks.0.attn.in_proj_bias', 'text.transformer.resblocks.0.attn.in_proj_weight', 'text.transformer.resblocks.0.attn.out_proj.bias', 'text.transformer.resblocks.0.attn.out_proj.weight', 'text.transformer.resblocks.0.ln_1.bias', 'text.transformer.resblocks.0.ln_1.weight', 'text.transformer.resblocks.0.ln_2.bias', 'text.transformer.resblocks.0.ln_2.weight', 'text.transformer.resblocks.0.mlp.c_fc.bias', 'text.transformer.resblocks.0.mlp.c_fc.weight', 'text.transformer.resblocks.0.mlp.c_proj.bias', 'text.transformer.resblocks.0.mlp.c_proj.weight', 'text.transformer.resblocks.1.attn.in_proj_bias', 'text.transformer.resblocks.1.attn.in_proj_weight', 'text.transformer.resblocks.1.attn.out_proj.bias', 'text.transformer.resblocks.1.attn.out_proj.weight', 'text.transformer.resblocks.1.ln_1.bias', 'text.transformer.resblocks.1.ln_1.w